In [1]:
import pandas as pd
from google.colab import files

# Carregar o arquivo Excel
uploaded = files.upload()

# Obtém o nome do arquivo carregado
file_name = list(uploaded.keys())[0]

# Lê o arquivo Excel
data = pd.read_excel(file_name)

# Remove a primeira linha que contém a descrição
data_cleaned = data.drop(0).reset_index(drop=True)

# Converte a coluna 'DATA' para o formato datetime
data_cleaned['DATA'] = pd.to_datetime(data_cleaned['DATA'], format='%d/%m/%Y %H:%M')

# Extrai a parte da data apenas para agrupar
data_cleaned['DATE'] = data_cleaned['DATA'].dt.date

# Função para calcular temperaturas diárias máximas, mínimas e médias
def calculate_daily_temperatures(df, location):
    # Converte a coluna de temperatura para numérico, forçando erros para NaN
    df[location] = pd.to_numeric(df[location], errors='coerce')

    # Agrupa por data e calcula a máxima, mínima e média
    daily_temps = df.groupby('DATE')[location].agg(['max', 'min', 'mean']).reset_index()
    daily_temps.columns = ['DATE', f'{location}_max', f'{location}_min', f'{location}_mean']

    return daily_temps

# Lista de locais (excluindo 'DATA' e 'DATE')
locations = data_cleaned.columns[1:-1]

# Calcula temperaturas diárias para cada local
daily_temperatures = [calculate_daily_temperatures(data_cleaned, loc) for loc in locations]

# Junta todos os resultados em um único dataframe
daily_temperatures_df = daily_temperatures[0]
for df in daily_temperatures[1:]:
    daily_temperatures_df = daily_temperatures_df.merge(df, on='DATE', how='outer')

# Salva os resultados em um novo arquivo Excel
output_file = 'daily_temperatures.xlsx'
daily_temperatures_df.to_excel(output_file, index=False)

# Download do arquivo
files.download(output_file)


Saving 2009_chat.xlsx to 2009_chat.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>